In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms, datasets

# ----------------------------
# הגדרות
# ----------------------------
num_classes = 10  # מספר מחלקות לדוגמה
checkpoint_path = "checkpoint.pth"  # נתיב ל-checkpoint אם קיים
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# טרנספורמציות לדאטא
# ----------------------------
train_transforms = transforms.Compose([
    transforms.Resize(224),  # ResNet דורש בדרך כלל 224x224
    transforms.ToTensor(),
])

train_ds = datasets.FakeData(transform=train_transforms)  # לדוגמה
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

# ----------------------------
# יצירת ResNet-18
# ----------------------------
backbone = models.resnet18(weights=None)
backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)  # התאמה למספר מחלקות
backbone = backbone.to(device)

# ----------------------------
# טעינת checkpoint אם קיים
# ----------------------------
try:
    checkpoint = torch.load(checkpoint_path)
    backbone.load_state_dict(checkpoint["model_state_dict"])
    print("Checkpoint loaded!")
except FileNotFoundError:
    print("No checkpoint found, training from scratch.")

# ----------------------------
# קפיאה של שכבות (fine-tuning)
# ----------------------------
# קפיאה של כל השכבות מלבד layer4 ו-fc
for name, param in backbone.named_parameters():
    if "layer4" not in name and "fc" not in name:
        param.requires_grad = False

# ----------------------------
# אובייקט optimizer (רק הפרמטרים הניתנים לאימון)
# ----------------------------
optimizer = optim.Adam(filter(lambda p: p.requires_grad, backbone.parameters()), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ----------------------------
# אימון קצר לדוגמה
# ----------------------------
backbone.train()
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    
    optimizer.zero_grad()
    outputs = backbone(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

print("Training step done!")
